In [ ]:
import pandas as pd
import numpy as np

# 1. 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# --- [핵심] 고급 전처리 함수 정의 ---
def advanced_preprocessing(df_train, df_test):
    # Train과 Test를 합쳐서 처리해야 일관성이 유지됩니다.
    # 단, 나중에 다시 분리하기 위해 구분자(is_train)를 둡니다.
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    df_test['completed'] = None # 타겟 컬럼 빈 값으로 생성
    
    # 두 데이터 합치기
    full_df = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)
    
    print("1. 이상치 처리 중...")

    # [이상치 처리] 이수 학기가 16을 넘으면 오타로 보고 중앙값(6)으로 대체
    median_semester = full_df[full_df['completed_semester'] <= 16]['completed_semester'].median()
    full_df.loc[full_df['completed_semester'] > 16, 'completed_semester'] = median_semester
    
    print("2. 파생 변수 생성 중...")

    # [텍스트 길이] 성실성 지표
    text_cols = ['whyBDA', 'what_to_gain', 'onedayclass_topic']
    for col in text_cols:
        full_df[f'{col}_len'] = full_df[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
    

    # [리스트 항목 수] 관심사 폭 지표
    list_cols = ['desired_certificate', 'desired_job', 'interested_company', 'incumbents_lecture_type']
    for col in list_cols:
        full_df[f'{col}_count'] = full_df[col].apply(lambda x: str(x).count(',') + 1 if pd.notnull(x) else 0)


    # [수강 이력 합계] 충성도 지표
    prev_cols = [c for c in full_df.columns if 'previous_class' in c]
    full_df['num_prev_classes'] = full_df[prev_cols].notnull().sum(axis=1)
    

    # [신청 분반 합계] 학습 의지 지표
    class_cols = ['class1', 'class2', 'class3', 'class4']
    full_df['num_classes'] = full_df[class_cols].notnull().sum(axis=1)
    

    print("3. 불필요 컬럼 및 희소 변수 정리 중...")
    # [컬럼 삭제] 
    # - generation: 9기(Train), 10기(Test)로 나뉘어 있어 변수로서 의미 없음 (오히려 방해됨)
    # - major1_1, major1_2: major_field가 있으므로 삭제 (너무 다양해서 과적합 유발)
    # - 결측치 90% 이상 컬럼들
    drop_cols = [
        'ID', 'generation', 'contest_award', 'idea_contest', 'contest_participation',
        'major1_1', 'major1_2', 
        'class3', 'class4', # num_classes로 대체했으므로 삭제
        'previous_class_3', 'previous_class_4', 'previous_class_5', 
        'previous_class_6', 'previous_class_7', 'previous_class_8' # num_prev_classes로 대체
    ]
    full_df = full_df.drop(columns=[c for c in drop_cols if c in full_df.columns])
    
    # [범주형 변수 일반화] - Train/Test 값 차이 해결!
    # major_field, school1 등 범주가 너무 많은 경우, 상위 N개만 남기고 나머지는 'Other'로 통합
    high_cardinality_cols = ['major_field', 'school1', 'desired_job', 'incumbents_company_level']
    
    for col in high_cardinality_cols:
        if col in full_df.columns:
            # 전체 데이터 기준 빈도수 계산
            val_counts = full_df[col].value_counts()
            # 10회 미만 등장하는 값은 'Other'로 변경 (일반화)
            valid_cats = val_counts[val_counts >= 10].index
            full_df[col] = full_df[col].apply(lambda x: x if x in valid_cats else 'Other')
            # 결측치는 'Unknown'으로 처리
            full_df[col] = full_df[col].fillna('Unknown')
            # 문자열로 통일 (school1이 숫자일 수 있으므로)
            full_df[col] = full_df[col].astype(str)

    # 나머지 범주형 변수 결측치 처리
    cat_cols = full_df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        full_df[col] = full_df[col].fillna('Unknown')

    # 수치형 변수 결측치 처리 (0 또는 중앙값)
    num_cols = full_df.select_dtypes(include=['float64', 'int64']).columns
    for col in num_cols:
        if col != 'completed': # 타겟 제외
            full_df[col] = full_df[col].fillna(0)

    # 다시 Train/Test 분리
    train_processed = full_df[full_df['is_train'] == 1].drop(columns=['is_train'])
    test_processed = full_df[full_df['is_train'] == 0].drop(columns=['is_train', 'completed'])
    
    # 타겟 타입 복구
    train_processed['completed'] = train_processed['completed'].astype(int)
    
    return train_processed, test_processed

# 전처리 실행
train_final, test_final = advanced_preprocessing(train, test)

print(f"최종 Train Shape: {train_final.shape}")
print(f"최종 Test Shape: {test_final.shape}")
print("전처리 완료! 이제 이 데이터로 모델을 학습하면 됩니다.")

1. 이상치 처리 중...
2. 파생 변수 생성 중...
3. 불필요 컬럼 및 희소 변수 정리 중...
최종 Train Shape: (748, 40)
최종 Test Shape: (814, 39)
전처리 완료! 이제 이 데이터로 모델을 학습하면 됩니다.


In [ ]:
# 1. 라이브러리 및 환경 설정

import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier

# 랜덤 시드 고정 (매번 실행 결과가 달라지지 않도록)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # 행운의 숫자 42


In [ ]:

# 2. 데이터 전처리 함수 (심화 버전)
def advanced_preprocessing(df_train, df_test):
    # 합쳐서 처리하기 위해 구분자 생성
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    df_test['completed'] = None 
    
    full_df = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)
    
    # 1) 이상치 보정 (16학기 초과 -> 중앙값 대체)
    median_semester = full_df[full_df['completed_semester'] <= 16]['completed_semester'].median()
    full_df.loc[full_df['completed_semester'] > 16, 'completed_semester'] = median_semester
    
    # 2) 파생 변수 생성
    # 텍스트 길이 (성실도)
    text_cols = ['whyBDA', 'what_to_gain', 'onedayclass_topic']
    for col in text_cols:
        full_df[f'{col}_len'] = full_df[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

    # [추가] 텍스트에서 '열정'이 보이는 키워드 찾기
    # 중요도가 높았던 '강의 규모 이유'와 '원데이 클래스 주제' 공략
    keywords = ['실무', '경험', '프로젝트', '분석', '취업', '데이터']

    for col in ['incumbents_lecture_scale_reason', 'onedayclass_topic', 'whyBDA']:
        for keyword in keywords:
            # 해당 키워드가 포함되면 1, 아니면 0
            full_df[f'{col}_has_{keyword}'] = full_df[col].apply(
                lambda x: 1 if keyword in str(x) else 0
            )
        
    # 리스트 항목 개수 (다양성)
    list_cols = ['desired_certificate', 'desired_job', 'interested_company', 'incumbents_lecture_type']
    for col in list_cols:
        full_df[f'{col}_count'] = full_df[col].apply(lambda x: str(x).count(',') + 1 if pd.notnull(x) else 0)

    # 수강 이력 및 신청 분반 수
    prev_cols = [c for c in full_df.columns if 'previous_class' in c]
    full_df['num_prev_classes'] = full_df[prev_cols].notnull().sum(axis=1)
    
    class_cols = ['class1', 'class2', 'class3', 'class4']
    full_df['num_classes'] = full_df[class_cols].notnull().sum(axis=1)
    
    # 3) 불필요 컬럼 삭제
    drop_cols = [
        'ID', 'generation', 'contest_award', 'idea_contest', 'contest_participation',
        'major1_1', 'major1_2', 'class3', 'class4'
    ] + prev_cols # 원본 이전 기수 컬럼들도 삭제 (개수 변수로 대체됨)
    
    full_df = full_df.drop(columns=[c for c in drop_cols if c in full_df.columns])
    
    # 4) 범주형 변수 처리 ('Other' 통합)
    # 범주가 너무 많은 컬럼은 상위 10개만 남기고 나머지는 'Other'로 묶음
    high_cardinality_cols = ['major_field', 'school1', 'desired_job', 'incumbents_company_level']
    for col in high_cardinality_cols:
        if col in full_df.columns:
            val_counts = full_df[col].value_counts()
            valid_cats = val_counts[val_counts >= 10].index # 빈도 10 이상만 유지
            full_df[col] = full_df[col].apply(lambda x: x if x in valid_cats else 'Other')
            full_df[col] = full_df[col].fillna('Unknown').astype(str)

    # 나머지 결측치 처리
    cat_cols = full_df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        full_df[col] = full_df[col].fillna('Unknown')
        
    num_cols = full_df.select_dtypes(include=['float64', 'int64']).columns
    for col in num_cols:
        if col != 'completed':
            full_df[col] = full_df[col].fillna(0)

    # 데이터 분리
    train_proc = full_df[full_df['is_train'] == 1].drop(columns=['is_train'])
    test_proc = full_df[full_df['is_train'] == 0].drop(columns=['is_train', 'completed'])
    train_proc['completed'] = train_proc['completed'].astype(int)
    
    return train_proc, test_proc

In [ ]:
# 3. 실행 및 모델링

# 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# 전처리 수행
train_final, test_final = advanced_preprocessing(train, test)


In [4]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 748 entries, 0 to 747
Data columns (total 58 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   school1                                   748 non-null    object 
 1   major type                                748 non-null    object 
 2   major_data                                748 non-null    bool   
 3   job                                       748 non-null    object 
 4   class1                                    748 non-null    int64  
 5   class2                                    748 non-null    float64
 6   re_registration                           748 non-null    object 
 7   nationality                               748 non-null    object 
 8   inflow_route                              748 non-null    object 
 9   whyBDA                                    748 non-null    object 
 10  what_to_gain                              7

In [5]:
test_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 814 entries, 748 to 1561
Data columns (total 57 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   school1                                   814 non-null    object 
 1   major type                                814 non-null    object 
 2   major_data                                814 non-null    bool   
 3   job                                       814 non-null    object 
 4   class1                                    814 non-null    int64  
 5   class2                                    814 non-null    float64
 6   re_registration                           814 non-null    object 
 7   nationality                               814 non-null    object 
 8   inflow_route                              814 non-null    object 
 9   whyBDA                                    814 non-null    object 
 10  what_to_gain                            

In [6]:

# 데이터 셋팅
target = 'completed'
X = train_final.drop(columns=[target])
y = train_final[target]
X_test = test_final.copy()

cat_features = [c for c in X.columns if X[c].dtype == 'object']

# 모델 학습 (Stratified K-Fold)
kf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))
"""
# CatBoost 파라미터
params = {
    'iterations': 2000,
    'learning_rate': 0.02, # 천천히 꼼꼼하게 학습
    'depth': 6,
    'loss_function': 'Logloss',
    'eval_metric': 'F1',
    'cat_features': cat_features,
    'early_stopping_rounds': 200,
    'verbose': 0, # 출력 끄기
    'random_seed': 42,
    'auto_class_weights': 'Balanced' # 불균형 데이터 자동 보정 (핵심!)
}"""

# 기존 params 대신 이걸로 교체!
# 과적합 방지에 집중한 설정입니다.
params = {
    'iterations': 10000,          # 충분히 많이
    'learning_rate': 0.005,       # 천천히 학습
    'depth': 4,                  # 깊이를 얕게 (핵심!)
    'l2_leaf_reg': 7,            # 규제 강화 (튀는 값 무시)
    'loss_function': 'Logloss',
    'eval_metric': 'F1',
    'cat_features': cat_features,
    'early_stopping_rounds': 300,
    'verbose': 0,
    'random_seed': 42,
    'auto_class_weights': 'Balanced',
    'bagging_temperature': 0.2   # 데이터 샘플링 다양화
}

print("모델 학습 시작...")
for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)
    
    oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
    test_preds += model.predict_proba(X_test)[:, 1] / 7

# ==========================================
# 4. Threshold(임계값) 최적화 및 제출
# ==========================================
best_th = 0.35
best_score = 0
for th in np.arange(0.2, 0.75, 0.001):
    pred = (oof_preds >= th).astype(int)
    score = f1_score(y, pred)
    if score > best_score:
        best_score = score
        best_th = th

print(f"최적 Threshold: {best_th:.2f}, CV F1 Score: {best_score:.4f}")

# 최종 예측
final_preds = (test_preds >= best_th).astype(int)

print(f"임계값 {best_th} 적용 시 1 예측 개수: {sum(final_preds)}")

submission['completed'] = final_preds
submission.to_csv('final_result.csv', index=False)
print("제출 파일 생성 완료: final_result.csv")

모델 학습 시작...
최적 Threshold: 0.50, CV F1 Score: 0.5009
임계값 0.5000000000000002 적용 시 1 예측 개수: 383
제출 파일 생성 완료: final_result.csv
